### SESIÓN 7: OPTIMIZACIÓN DE LOS PARÁMETROS DE UN MODELO USANDO ALGORITMOS GENÉTICOS.

- Carmen Miguel Spínola
- Miguel Ángel Molina de la Rosa

#### OPTIMIZACIÓN DE HÉLICE

In [ ]:
from modelo.helice import *

##### Definición del cromosoma:

Cada cromosoma será un vector de genes binarios. Estos tienen distinta longitud según la cantidad máxima que deben codificar:

- **Omega (10 bits):** Técnicamente se podrían codificar los números del 0 al 200 con 8 bits, teniendo 256 codificaciones posibles y espacio para algunos valores decimales. Sin embargo, con 10 bits se pueden representar más números reales, teniendo valores que van aumentando desde el principio del rango al final según:

    - 2^10 = 1024 combinaciones posibles → 200 / 1023 = 0.195 rad/s.
    - Frente a:
    - 2^8 = 256 combinaciones posibles → 200 / 255 = 0.784 rad/s.

    De esta manera, se requiere un pequeño espacio más de memoria, pero se obtiene más precisión.

- **R (10 bits):** De la misma manera, se codifica un conjunto continuo de valores de 0.1 a 2 metros. Como la precisión puede afectar en la búsqueda del óptimo, se opta por una precisión alta con una variación entre codificaciones contiguas de:

    - 2^10 = 1024 combinaciones posibles → (2 - 0.1) / 1023 = 0.0.00186 metros.

- **Número de palas, b (2 bits):** Codifica valores enteros que oscilan entre 2 y 5. No necesitamos preocuparnos de la precisión y, como necesitamos representar 4 valores, usamos 2 bits (4 combinaciones posibles), de manera que:
    - `00 = 2 palas`
    - `01 = 3 palas`
    - `10 = 4 palas`
    - `11 = 5 palas`.

- **Theta0 (5 bits):** El conjunto continuo de valores nos obliga a ajustarnos a una precisión aceptable. Se elige codificar el rango físico de -0.26 a 0.26 (52 valores) con 5 bits:
    - 2^5 = 32 combinaciones posibles → 0.52 / 31 = 0.0168 radianes.

- **Parámetro de torsión p (8 bits):** Se necesita más precisión al tener que representar un rango mayor de valores físicos (25 valores). Se adjudican 8 bits al gen de p:
    - 2^8 = 256 combinaciones posibles → p = 25 / 255 = 0.098 grados.

- **Cuerda, anchura de la pala (8 bits):** Se necesita codificar un rango físico de entre 0.01 y 0.2 metros (0.19 metros es la diferencia entre el máximo y el mínimo valor). La variación real de los metros es muy pequeña, por lo que se eligen 8 bits para una gran precisión:
    - 2^8 = 256 combinaciones posibles → cuerda = 0.19 / 1023 = 0.000186 metros.

Tendremos finalmente un cromosoma de 43 bits con genes binarios de distinto número de bits.


##### Programas de optimización de parámetros del problema: 

- **Maximizar la tracción T (manteniendo una tracción mínima de 30 Newtons) con el mínimo radio posible R:**

In [ ]:
import random

# Función para convertir un cromosoma binario a un número decimal con un rango específico
def binario_a_decimal_rango(cromosoma, min_valor, max_valor):
    decimal = int("".join(str(x) for x in cromosoma), 2)
    rango = max_valor - min_valor
    valor = min_valor + (decimal % (rango + 1))
    return valor

# Función de cruce 
def fun_cruzar(cromosoma1, cromosoma2):
    l1 = len(cromosoma1)
    l2 = len(cromosoma2)
    cruce1 = cromosoma1[0:l1//2] + cromosoma2[l1//2:l2]
    cruce2 = cromosoma2[0:l2//2] + cromosoma1[l2//2:l1]
    return [cruce1, cruce2]

# Función de mutación
def fun_mutar(cromosoma, prob):
    for i in range(len(cromosoma)):
        if random.uniform(0, 1) < prob:
            cromosoma[i] = 1 - cromosoma[i]  # Cambia el bit de 0 a 1 o de 1 a 0
    return cromosoma


def fun_fitness(cromosoma):
    # Decodificación de los parámetros
    omega = binario_a_decimal_rango(cromosoma[0:10], 0, 200)  # Omega (rad/s)
    R = binario_a_decimal_rango(cromosoma[10:20], 0.1, 2)     # Radio (m)
    b = binario_a_decimal_rango(cromosoma[20:22], 2, 5)       # Número de palas
    theta0 = binario_a_decimal_rango(cromosoma[22:27], -0.26, 0.26)  # Ángulo de paso colectivo (radianes)
    p = binario_a_decimal_rango(cromosoma[27:35], 0, 25)      # Torsión (grados)
    cuerda = binario_a_decimal_rango(cromosoma[35:43], 0.01, 0.2)  # Anchura de la pala (m)
    
    # Valores fijos
    vz = 70  # Velocidad de vuelo fija (m/s)
    h = 1000  # Altura fija (m)

    print(f"\nEvaluando cromosoma: {cromosoma}")
    print(f"Parámetros decodificados: omega={omega}, R={R}, b={b}, theta0={theta0}, p={p}, cuerda={cuerda}")

    try:
        # Calcular hélice con los parámetros decodificados
        T, P, efic, mach_punta = calcular_helice(omega, vz, R, b, h, theta0=theta0, tors_param=['h',p], chord_params=cuerda)
        print(f"Resultados de calcular_helice: T={T}, P={P}, efic={efic}, mach_punta={mach_punta}")
       
        # Fitness basado en maximizar la tracción(minimo 30N) y minimizar el radio de las palas
        if T < 30:  # Penalización por tracción insuficiente
            fitness = 0
            print(f"Tracción insuficiente T = {T} N, fitness penalizado a 0")
        else:
            fitness = T - 0.1 * R # Calculamos el fitness como la tracción menos 0.1 veces el radio (a más T más fitness y a más radio menos fitnnes).
            print(f"Fitness calculado: {fitness}")
    except Exception as e:
        print(f"Error en calcular_helice: {e}")
        fitness = 0
    return fitness


class ProblemaGenetico:
    def __init__(self, valores_posibles, funcion_decodificar, funcion_mutar, funcion_cruzar, funcion_fitness, tamano_poblacion, longitud_cromosoma):
        self.valores_posibles = valores_posibles
        self.funcion_decodificar = funcion_decodificar
        self.funcion_mutar = funcion_mutar
        self.funcion_cruzar = funcion_cruzar
        self.funcion_fitness = funcion_fitness
        self.tamano_poblacion = tamano_poblacion
        self.longitud_cromosoma = longitud_cromosoma  
        self.poblacion = self.crear_poblacion()
    
    def crear_poblacion(self):
        poblacion = []
        for _ in range(self.tamano_poblacion):
            cromosoma = [random.choice(self.valores_posibles) for _ in range(self.longitud_cromosoma)]
            poblacion.append(cromosoma)
        return poblacion
    
    def evaluar_poblacion(self):
        fitnesses = []
        for cromosoma in self.poblacion:
            fitness = self.funcion_fitness(cromosoma)
            fitnesses.append(fitness)
        return fitnesses
    
    def seleccionar(self, fitnesses):
        poblacion_ordenada = sorted(zip(self.poblacion, fitnesses), key=lambda x: x[1], reverse=True)
        mejores_parejas = [cromosoma for cromosoma, _ in poblacion_ordenada[:len(self.poblacion)//2]]
        
        if len(mejores_parejas) % 2 != 0:
            mejores_parejas.append(mejores_parejas[-1])
        
        return mejores_parejas
    
    def cruzar(self, mejores_parejas):
        nueva_poblacion = []
        for i in range(0, len(mejores_parejas), 2):
            cromosoma1 = mejores_parejas[i]
            cromosoma2 = mejores_parejas[i + 1]
            hijos = self.funcion_cruzar(cromosoma1, cromosoma2)
            nueva_poblacion.extend(hijos)
        return nueva_poblacion
    
    def mutar(self, nueva_poblacion, prob_mutacion):
        return [self.funcion_mutar(cromosoma, prob_mutacion) for cromosoma in nueva_poblacion]
    
    def resolver(self, generaciones, prob_mutacion):
        for generacion in range(generaciones):
            print(f"\n--- Generación {generacion + 1} ---")
            fitnesses = self.evaluar_poblacion()
            print(f"Fitnesses de la población: {fitnesses}")
            print(f"Mejor fitness en esta generación: {max(fitnesses)}")
            
            mejores_parejas = self.seleccionar(fitnesses)
            nueva_poblacion = self.cruzar(mejores_parejas)
            nueva_poblacion = self.mutar(nueva_poblacion, prob_mutacion)
            
            self.poblacion = nueva_poblacion[:self.tamano_poblacion]

        # Evaluamos la población final
        fitnesses = self.evaluar_poblacion()
        mejor_fitness = max(fitnesses)
        indice_mejor = fitnesses.index(mejor_fitness)
        mejor_cromosoma = self.poblacion[indice_mejor]
        print("\nMejor cromosoma final:")
        print(f"Cromosoma: {mejor_cromosoma} - Fitness: {mejor_fitness}")


longitud_cromosoma = 43
valores_posibles = [0, 1]  # Suposición de valores posibles para los cromosomas (0 o 1)
problema = ProblemaGenetico(valores_posibles, binario_a_decimal_rango, fun_mutar, fun_cruzar, fun_fitness, 10, longitud_cromosoma)

# Resolver el problema durante 100 generaciones con probabilidad de mutación de 0.1
problema.resolver(100, 0.1)


Se ha plantado la resolución del problema para un número de generaciones y una probabilidad de mutación. No hay probabilidad de cruce en esta primera implementación (los cromosomas se cruzarán siempre).
Al tener una penalización excesiva (el fitness de cierto cromosoma será 0 si la tracción no llega a 30) y una función de cruce destructiva en la que los hijos pasan a la siguiente generación aunque sean 'peores' que los padres, al final del cómputo es posible que obtengamos un cromosoma con un fitness de 0 o menor de menor valor que los calculados en generaciones anteriores.

En la siguiente implementación se prueba con una penalización más suave en caso de no llegar a la tracción mínima.

In [ ]:
import random

# Función para convertir un cromosoma binario a un número decimal con un rango específico
def binario_a_decimal_rango(cromosoma, min_valor, max_valor):
    decimal = int("".join(str(x) for x in cromosoma), 2)
    rango = max_valor - min_valor
    valor = min_valor + (decimal % (rango + 1))
    return valor

# Función de cruce 
def fun_cruzar(cromosoma1, cromosoma2):
    l1 = len(cromosoma1)
    l2 = len(cromosoma2)
    cruce1 = cromosoma1[0:l1//2] + cromosoma2[l1//2:l2]
    cruce2 = cromosoma2[0:l2//2] + cromosoma1[l2//2:l1]
    return [cruce1, cruce2]

# Función de mutación
def fun_mutar(cromosoma, prob):
    for i in range(len(cromosoma)):
        if random.uniform(0, 1) < prob:
            cromosoma[i] = 1 - cromosoma[i]  # Cambia el bit de 0 a 1 o de 1 a 0
    return cromosoma


def fun_fitness(cromosoma):
    # Decodificación de los parámetros
    omega = binario_a_decimal_rango(cromosoma[0:10], 0, 200)  # Omega (rad/s)
    R = binario_a_decimal_rango(cromosoma[10:20], 0.1, 2)     # Radio (m)
    b = binario_a_decimal_rango(cromosoma[20:22], 2, 5)       # Número de palas
    theta0 = binario_a_decimal_rango(cromosoma[22:27], -0.26, 0.26)  # Ángulo de paso colectivo (radianes)
    p = binario_a_decimal_rango(cromosoma[27:35], 0, 25)      # Torsión (grados)
    cuerda = binario_a_decimal_rango(cromosoma[35:43], 0.01, 0.2)  # Anchura de la pala (m)
    
    # Valores fijos
    vz = 70  # Velocidad de vuelo fija (m/s)
    h = 1000  # Altura fija (m)

    print(f"\nEvaluando cromosoma: {cromosoma}")
    print(f"Parámetros decodificados: omega={omega}, R={R}, b={b}, theta0={theta0}, p={p}, cuerda={cuerda}")

    try:
        # Calcular hélice con los parámetros decodificados
        T, P, efic, mach_punta = calcular_helice(omega, vz, R, b, h, theta0=theta0, tors_param=['h',p], chord_params=cuerda)
        print(f"Resultados de calcular_helice: T={T}, P={P}, efic={efic}, mach_punta={mach_punta}")
       
        # Fitness basado en maximizar la tracción(minimo 30N) y minimizar el radio de las palas
        if T < 30:
            fitness = T * 0.5  # Penalización menos severa
            print(f"Tracción insuficiente T = {T} N, fitness penalizado a la mitad")
        else:
            fitness = T - 0.1 * R # Calculamos el fitness como la tracción menos 0.1 veces el radio (a más T más fitness y a más radio menos fitnnes).
            print(f"Fitness calculado: {fitness}")
    except Exception as e:
        print(f"Error en calcular_helice: {e}")
        fitness = 0
    return fitness


class ProblemaGenetico:
    def __init__(self, valores_posibles, funcion_decodificar, funcion_mutar, funcion_cruzar, funcion_fitness, tamano_poblacion, longitud_cromosoma):
        self.valores_posibles = valores_posibles
        self.funcion_decodificar = funcion_decodificar
        self.funcion_mutar = funcion_mutar
        self.funcion_cruzar = funcion_cruzar
        self.funcion_fitness = funcion_fitness
        self.tamano_poblacion = tamano_poblacion
        self.longitud_cromosoma = longitud_cromosoma  
        self.poblacion = self.crear_poblacion()
    
    def crear_poblacion(self):
        poblacion = []
        for _ in range(self.tamano_poblacion):
            cromosoma = [random.choice(self.valores_posibles) for _ in range(self.longitud_cromosoma)]
            poblacion.append(cromosoma)
        return poblacion
    
    def evaluar_poblacion(self):
        fitnesses = []
        for cromosoma in self.poblacion:
            fitness = self.funcion_fitness(cromosoma)
            fitnesses.append(fitness)
        return fitnesses
    
    def seleccionar(self, fitnesses):
        poblacion_ordenada = sorted(zip(self.poblacion, fitnesses), key=lambda x: x[1], reverse=True)
        mejores_parejas = [cromosoma for cromosoma, _ in poblacion_ordenada[:len(self.poblacion)//2]]
        
        if len(mejores_parejas) % 2 != 0:
            mejores_parejas.append(mejores_parejas[-1])
        
        return mejores_parejas
    
    def cruzar(self, mejores_parejas):
        nueva_poblacion = []
        for i in range(0, len(mejores_parejas), 2):
            cromosoma1 = mejores_parejas[i]
            cromosoma2 = mejores_parejas[i + 1]
            hijos = self.funcion_cruzar(cromosoma1, cromosoma2)
            nueva_poblacion.extend(hijos)
        return nueva_poblacion
    
    def mutar(self, nueva_poblacion, prob_mutacion):
        return [self.funcion_mutar(cromosoma, prob_mutacion) for cromosoma in nueva_poblacion]
    
    def resolver(self, generaciones, prob_mutacion):
        for generacion in range(generaciones):
            print(f"\n--- Generación {generacion + 1} ---")
            fitnesses = self.evaluar_poblacion()
            print(f"Fitnesses de la población: {fitnesses}")
            print(f"Mejor fitness en esta generación: {max(fitnesses)}")
            
            mejores_parejas = self.seleccionar(fitnesses)
            nueva_poblacion = self.cruzar(mejores_parejas)
            nueva_poblacion = self.mutar(nueva_poblacion, prob_mutacion)
            
            self.poblacion = nueva_poblacion[:self.tamano_poblacion]

        # Evaluamos la población final
        fitnesses = self.evaluar_poblacion()
        mejor_fitness = max(fitnesses)
        indice_mejor = fitnesses.index(mejor_fitness)
        mejor_cromosoma = self.poblacion[indice_mejor]
        print("\nMejor cromosoma final:")
        print(f"Cromosoma: {mejor_cromosoma} - Fitness: {mejor_fitness}")


longitud_cromosoma = 43
valores_posibles = [0, 1]  # Suposición de valores posibles para los cromosomas (0 o 1)
problema = ProblemaGenetico(valores_posibles, binario_a_decimal_rango, fun_mutar, fun_cruzar, fun_fitness, 10, longitud_cromosoma)

# Resolver el problema durante 100 generaciones con probabilidad de mutación de 0.1
problema.resolver(100, 0.1)


En este caso, evitamos que el fitness final sea 0. Esta implementación es, sin embargo, poco óptima en cuanto a lo que se pide en el enunciado: estamos obteniendo un valor máximo de tracción con un valor mínimo para el radio de las palas haciendo caso omiso de la restricción inicial.

La solución entonces pasa por hacer una selección más elitista y evitar que el cruce sea destructivo añadiendo a la siguiente generación solo a los hijos que tengan mayor fitness que sus padres.

In [ ]:
import random

# Función para convertir un cromosoma binario a un número decimal con un rango específico
def binario_a_decimal_rango(cromosoma, min_valor, max_valor):
    decimal = int("".join(str(x) for x in cromosoma), 2)
    rango = max_valor - min_valor
    valor = min_valor + (decimal % (rango + 1))
    return valor

# Función de cruce
def fun_cruzar(cromosoma1, cromosoma2):
    l1 = len(cromosoma1)
    l2 = len(cromosoma2)
    cruce1 = cromosoma1[0:l1//2] + cromosoma2[l1//2:l2]
    cruce2 = cromosoma2[0:l2//2] + cromosoma1[l2//2:l1]
    return [cruce1, cruce2]

# Función de mutación
def fun_mutar(cromosoma, prob):
    for i in range(len(cromosoma)):
        if random.uniform(0, 1) < prob:
            cromosoma[i] = 1 - cromosoma[i]  # Cambia el bit de 0 a 1 o de 1 a 0
    return cromosoma

# Función de fitness
def fun_fitness(cromosoma):
    # Decodificación de los parámetros
    omega = binario_a_decimal_rango(cromosoma[0:10], 0, 200)  # Omega (rad/s)
    R = binario_a_decimal_rango(cromosoma[10:20], 0.1, 2)     # Radio (m)
    b = binario_a_decimal_rango(cromosoma[20:22], 2, 5)       # Número de palas
    theta0 = binario_a_decimal_rango(cromosoma[22:27], -0.26, 0.26)  # Ángulo de paso colectivo (radianes)
    p = binario_a_decimal_rango(cromosoma[27:35], 0, 25)      # Torsión (grados)
    cuerda = binario_a_decimal_rango(cromosoma[35:43], 0.01, 0.2)  # Anchura de la pala (m)
    
    # Valores fijos
    vz = 70  # Velocidad de vuelo fija (m/s)
    h = 1000  # Altura fija (m)

    print(f"\nEvaluando cromosoma: {cromosoma}")
    print(f"Parámetros decodificados: omega={omega}, R={R}, b={b}, theta0={theta0}, p={p}, cuerda={cuerda}")

    try:
        # Calcular hélice con los parámetros decodificados
        T, P, efic, mach_punta = calcular_helice(omega, vz, R, b, h, theta0=theta0, tors_param=['h',p], chord_params=cuerda)
        print(f"Resultados de calcular_helice: T={T}, P={P}, efic={efic}, mach_punta={mach_punta}")
       
        # Fitness basado en maximizar la tracción(minimo 30N) y minimizar el radio de las palas
        if T < 30:
            fitness = 0  # Penalización por tracción insuficiente
            print(f"Tracción insuficiente T = {T} N, fitness penalizado a la mitad")
        else:
            fitness = T - 0.1 * R # Calculamos el fitness como la tracción menos 0.1 veces el radio (a más T más fitness y a más radio menos fitnnes).
            print(f"Fitness calculado: {fitness}")
    except Exception as e:
        print(f"Error en calcular_helice: {e}")
        fitness = 0
    return fitness


class ProblemaGenetico:
    def __init__(self, valores_posibles, funcion_decodificar, funcion_mutar, funcion_cruzar, funcion_fitness, tamano_poblacion, longitud_cromosoma):
        self.valores_posibles = valores_posibles
        self.funcion_decodificar = funcion_decodificar
        self.funcion_mutar = funcion_mutar
        self.funcion_cruzar = funcion_cruzar
        self.funcion_fitness = funcion_fitness
        self.tamano_poblacion = tamano_poblacion
        self.longitud_cromosoma = longitud_cromosoma  
        self.poblacion = self.crear_poblacion()
    
    def crear_poblacion(self):
        poblacion = []
        for _ in range(self.tamano_poblacion):
            cromosoma = [random.choice(self.valores_posibles) for _ in range(self.longitud_cromosoma)]
            poblacion.append(cromosoma)
        return poblacion
    
    def evaluar_poblacion(self):
        fitnesses = []
        for cromosoma in self.poblacion:
            fitness = self.funcion_fitness(cromosoma)
            fitnesses.append(fitness)
        return fitnesses
    
    def seleccionar(self, fitnesses):
        poblacion_ordenada = sorted(zip(self.poblacion, fitnesses), key=lambda x: x[1], reverse=True)
        mejores_parejas = [cromosoma for cromosoma, _ in poblacion_ordenada[:len(self.poblacion)//2]]
        
        if len(mejores_parejas) % 2 != 0:
            mejores_parejas.append(mejores_parejas[-1])
        
        return mejores_parejas
    
    #Implementación de la función de cruce no destructiva
    def cruzar(self, mejores_parejas):
        nueva_poblacion = []
        for i in range(0, len(mejores_parejas), 2):
            cromosoma1 = mejores_parejas[i]
            cromosoma2 = mejores_parejas[i + 1]
            hijos = self.funcion_cruzar(cromosoma1, cromosoma2)
            
            # Calcula el fitness de los hijos y de los padres
            fitness_padre1 = self.funcion_fitness(cromosoma1)
            fitness_padre2 = self.funcion_fitness(cromosoma2)
            fitness_hijo1 = self.funcion_fitness(hijos[0])
            fitness_hijo2 = self.funcion_fitness(hijos[1])
            
            # Solo se conserva el hijo si tiene mejor fitness que ambos padres
            if fitness_hijo1 > fitness_padre1 and fitness_hijo1 > fitness_padre2:
                nueva_poblacion.append(hijos[0])
            else:
                nueva_poblacion.append(cromosoma1) #si el hijo1 no supera el fitness de ambos padres, se conserva el padre1
            
            if fitness_hijo2 > fitness_padre1 and fitness_hijo2 > fitness_padre2:
                nueva_poblacion.append(hijos[1])
            else:
                nueva_poblacion.append(cromosoma2) #si el hijo2 no supera el fitness de ambos padres, se conserva el padre2
        
        return nueva_poblacion
    
    def mutar(self, nueva_poblacion, prob_mutacion):
        return [self.funcion_mutar(cromosoma, prob_mutacion) for cromosoma in nueva_poblacion]
    
    def resolver(self, generaciones, prob_mutacion):
        for generacion in range(generaciones):
            print(f"\n--- Generación {generacion + 1} ---")
            fitnesses = self.evaluar_poblacion()
            print(f"Fitnesses de la población: {fitnesses}")
            print(f"Mejor fitness en esta generación: {max(fitnesses)}")
            
            mejores_parejas = self.seleccionar(fitnesses)
            nueva_poblacion = self.cruzar(mejores_parejas)
            nueva_poblacion = self.mutar(nueva_poblacion, prob_mutacion)
            
            self.poblacion = nueva_poblacion[:self.tamano_poblacion]

        # Evaluamos la población final
        fitnesses = self.evaluar_poblacion()
        mejor_fitness = max(fitnesses)
        indice_mejor = fitnesses.index(mejor_fitness)
        mejor_cromosoma = self.poblacion[indice_mejor]
        print("\nMejor cromosoma final:")
        print(f"Cromosoma: {mejor_cromosoma} - Fitness: {mejor_fitness}")


longitud_cromosoma = 43
valores_posibles = [0, 1]  # Suposición de valores posibles para los cromosomas (0 o 1)
problema = ProblemaGenetico(valores_posibles, binario_a_decimal_rango, fun_mutar, fun_cruzar, fun_fitness, 10, longitud_cromosoma)

# Resolver el problema durante 100 generaciones con probabilidad de mutación de 0.1
problema.resolver(100, 0.1)


En este caso, aunque la penalización en caso de que la T sea menor que 30 es de un fitness de 0, si se han obtenido cromosomas con un fitness positivo en generaciones anteriores, la actual no tendrá un fitness de 0 al escoger los hijos solo cuando tienen un fitness mayor que su respectivo padre y, por tanto, el cromosoma elegido finalmente nunca tendrá un fitness de 0.
Se sacrifica la posibilidad de que tras una modificación de los genes desde un padre con fitness positivo, pasando por un hijo con fitnes 0 o peor que el suyo, se obtenga un cromosoma con un fitness mejor que ese padre en generaaciones posteriores. Por otro lado, evitamos el hecho de que se obtenga al final de la ejecución un cromosoma inválido que no cumple las restricciones del enunciado.

- **Maximizar la eficiencia cuando vuela a 70 m/s:**

En el anterior ejemplo, hemos probado con una función de fitness algo más compleja. Ahora tenemos una función sin penalizaciones y daremos más importancia a la modificación del resto de funciones.

In [ ]:
import random

# Función para convertir un cromosoma binario a un número decimal con un rango específico
def binario_a_decimal_rango(cromosoma, min_valor, max_valor):
    decimal = int("".join(str(x) for x in cromosoma), 2)
    rango = max_valor - min_valor
    valor = min_valor + (decimal % (rango + 1))
    return valor

# Función de cruce 
def fun_cruzar(cromosoma1, cromosoma2):
    l1 = len(cromosoma1)
    l2 = len(cromosoma2)
    cruce1 = cromosoma1[0:l1//2] + cromosoma2[l1//2:l2]
    cruce2 = cromosoma2[0:l2//2] + cromosoma1[l2//2:l1]
    return [cruce1, cruce2]

# Función de mutación
def fun_mutar(cromosoma, prob):
    for i in range(len(cromosoma)):
        if random.uniform(0, 1) < prob:
            cromosoma[i] = 1 - cromosoma[i]  # Cambia el bit de 0 a 1 o de 1 a 0
    return cromosoma


def fun_fitness(cromosoma):
    # Decodificación de los parámetros
    omega = binario_a_decimal_rango(cromosoma[0:10], 0, 200)  # Omega (rad/s)
    R = binario_a_decimal_rango(cromosoma[10:20], 0.1, 2)     # Radio (m)
    b = binario_a_decimal_rango(cromosoma[20:22], 2, 5)       # Número de palas
    theta0 = binario_a_decimal_rango(cromosoma[22:27], -0.26, 0.26)  # Ángulo de paso colectivo (radianes)
    p = binario_a_decimal_rango(cromosoma[27:35], 0, 25)      # Torsión (grados)
    cuerda = binario_a_decimal_rango(cromosoma[35:43], 0.01, 0.2)  # Anchura de la pala (m)
    
    # Valores fijos
    vz = 70  # Velocidad de vuelo fija (m/s)
    h = 1000  # Altura fija (m)

    print(f"\nEvaluando cromosoma: {cromosoma}")
    print(f"Parámetros decodificados: omega={omega}, R={R}, b={b}, theta0={theta0}, p={p}, cuerda={cuerda}")

    try:
        # Calcular hélice con los parámetros decodificados
        T, P, efic, mach_punta = calcular_helice(omega, vz, R, b, h, theta0=theta0, tors_param=['h',p], chord_params=cuerda)
        print(f"Resultados de calcular_helice: T={T}, P={P}, efic={efic}, mach_punta={mach_punta}")
       
        # Fitness basado en maximizar la eficiencia
        fitness = efic*10 # Calculamos el fitness como la propia eficiencia, se ha multiplicado para que no salgan valores muy pequeños (por comodidad)
        print(f"Fitness calculado: {fitness}")
    except Exception as e:
        print(f"Error en calcular_helice: {e}")
        fitness = 0
    return fitness


class ProblemaGenetico:
    def __init__(self, valores_posibles, funcion_decodificar, funcion_mutar, funcion_cruzar, funcion_fitness, tamano_poblacion, longitud_cromosoma):
        self.valores_posibles = valores_posibles
        self.funcion_decodificar = funcion_decodificar
        self.funcion_mutar = funcion_mutar
        self.funcion_cruzar = funcion_cruzar
        self.funcion_fitness = funcion_fitness
        self.tamano_poblacion = tamano_poblacion
        self.longitud_cromosoma = longitud_cromosoma  
        self.poblacion = self.crear_poblacion()
    
    def crear_poblacion(self):
        poblacion = []
        for _ in range(self.tamano_poblacion):
            cromosoma = [random.choice(self.valores_posibles) for _ in range(self.longitud_cromosoma)]
            poblacion.append(cromosoma)
        return poblacion
    
    def evaluar_poblacion(self):
        fitnesses = []
        for cromosoma in self.poblacion:
            fitness = self.funcion_fitness(cromosoma)
            fitnesses.append(fitness)
        return fitnesses
    
    def seleccionar(self, fitnesses):
        poblacion_ordenada = sorted(zip(self.poblacion, fitnesses), key=lambda x: x[1], reverse=True)
        mejores_parejas = [cromosoma for cromosoma, _ in poblacion_ordenada[:len(self.poblacion)//2]]
        
        if len(mejores_parejas) % 2 != 0:
            mejores_parejas.append(mejores_parejas[-1])
        
        return mejores_parejas
    
    def cruzar(self, mejores_parejas):
        nueva_poblacion = []
        for i in range(0, len(mejores_parejas), 2):
            cromosoma1 = mejores_parejas[i]
            cromosoma2 = mejores_parejas[i + 1]
            hijos = self.funcion_cruzar(cromosoma1, cromosoma2)
            nueva_poblacion.extend(hijos)
        return nueva_poblacion
    
    def mutar(self, nueva_poblacion, prob_mutacion):
        return [self.funcion_mutar(cromosoma, prob_mutacion) for cromosoma in nueva_poblacion]
    
    def resolver(self, generaciones, prob_mutacion):
        for generacion in range(generaciones):
            print(f"\n--- Generación {generacion + 1} ---")
            fitnesses = self.evaluar_poblacion()
            print(f"Fitnesses de la población: {fitnesses}")
            print(f"Mejor fitness en esta generación: {max(fitnesses)}")
            
            mejores_parejas = self.seleccionar(fitnesses)
            nueva_poblacion = self.cruzar(mejores_parejas)
            nueva_poblacion = self.mutar(nueva_poblacion, prob_mutacion)
            
            self.poblacion = nueva_poblacion[:self.tamano_poblacion]

        # Evaluamos la población final
        fitnesses = self.evaluar_poblacion()
        mejor_fitness = max(fitnesses)
        indice_mejor = fitnesses.index(mejor_fitness)
        mejor_cromosoma = self.poblacion[indice_mejor]
        print("\nMejor cromosoma final:")
        print(f"Cromosoma: {mejor_cromosoma} - Fitness: {mejor_fitness}")


longitud_cromosoma = 43
valores_posibles = [0, 1]  # Suposición de valores posibles para los cromosomas (0 o 1)
problema = ProblemaGenetico(valores_posibles, binario_a_decimal_rango, fun_mutar, fun_cruzar, fun_fitness, 10, longitud_cromosoma)

# Resolver el problema durante 100 generaciones con probabilidad de mutación de 0.1
problema.resolver(100, 0.1)


Se ha plantado la resolución del problema para un número de generaciones y una probabilidad de mutación. No hay probabilidad de cruce en esta primera implementación (los cromosomas se cruzarán siempre).
La función de cruce es destructiva (los hijos pasarán a la siguiente generación independientemente de si su fitness es mejor que el de sus padres o no). Esto provoca que finalmente obtengamos un fitness más bajo para el cromosoma final del que hemos ido obteniendo en generaciones anteriores.

Se prueba en la siguiente implementación con una función no destructiva: se comprueba previamente si los hijos tienen mejor fitness que sus padres antes de que pasen a la siguiente generación.

In [ ]:
import random

# Función para convertir un cromosoma binario a un número decimal con un rango específico
def binario_a_decimal_rango(cromosoma, min_valor, max_valor):
    decimal = int("".join(str(x) for x in cromosoma), 2)
    rango = max_valor - min_valor
    valor = min_valor + (decimal % (rango + 1))
    return valor

# Función de cruce
def fun_cruzar(cromosoma1, cromosoma2):
    l1 = len(cromosoma1)
    l2 = len(cromosoma2)
    cruce1 = cromosoma1[0:l1//2] + cromosoma2[l1//2:l2]
    cruce2 = cromosoma2[0:l2//2] + cromosoma1[l2//2:l1]
    return [cruce1, cruce2]

# Función de mutación
def fun_mutar(cromosoma, prob):
    for i in range(len(cromosoma)):
        if random.uniform(0, 1) < prob:
            cromosoma[i] = 1 - cromosoma[i]  # Cambia el bit de 0 a 1 o de 1 a 0
    return cromosoma

# Función de fitness
def fun_fitness(cromosoma):
    # Decodificación de los parámetros
    omega = binario_a_decimal_rango(cromosoma[0:10], 0, 200)  # Omega (rad/s)
    R = binario_a_decimal_rango(cromosoma[10:20], 0.1, 2)     # Radio (m)
    b = binario_a_decimal_rango(cromosoma[20:22], 2, 5)       # Número de palas
    theta0 = binario_a_decimal_rango(cromosoma[22:27], -0.26, 0.26)  # Ángulo de paso colectivo (radianes)
    p = binario_a_decimal_rango(cromosoma[27:35], 0, 25)      # Torsión (grados)
    cuerda = binario_a_decimal_rango(cromosoma[35:43], 0.01, 0.2)  # Anchura de la pala (m)
    
    # Valores fijos
    vz = 70  # Velocidad de vuelo fija (m/s)
    h = 1000  # Altura fija (m)

    print(f"\nEvaluando cromosoma: {cromosoma}")
    print(f"Parámetros decodificados: omega={omega}, R={R}, b={b}, theta0={theta0}, p={p}, cuerda={cuerda}")

    try:
        # Calcular hélice con los parámetros decodificados
        T, P, efic, mach_punta = calcular_helice(omega, vz, R, b, h, theta0=theta0, tors_param=['h',p], chord_params=cuerda)
        print(f"Resultados de calcular_helice: T={T}, P={P}, efic={efic}, mach_punta={mach_punta}")
       
        # Fitness basado en maximizar la eficiencia
        fitness = efic*10 # Calculamos el fitness como la propia eficiencia, se ha multiplicado para que no salgan valores muy pequeños (por comodidad)
        print(f"Fitness calculado: {fitness}")
    except Exception as e:
        print(f"Error en calcular_helice: {e}")
        fitness = 0
    return fitness


class ProblemaGenetico:
    def __init__(self, valores_posibles, funcion_decodificar, funcion_mutar, funcion_cruzar, funcion_fitness, tamano_poblacion, longitud_cromosoma):
        self.valores_posibles = valores_posibles
        self.funcion_decodificar = funcion_decodificar
        self.funcion_mutar = funcion_mutar
        self.funcion_cruzar = funcion_cruzar
        self.funcion_fitness = funcion_fitness
        self.tamano_poblacion = tamano_poblacion
        self.longitud_cromosoma = longitud_cromosoma  
        self.poblacion = self.crear_poblacion()
    
    def crear_poblacion(self):
        poblacion = []
        for _ in range(self.tamano_poblacion):
            cromosoma = [random.choice(self.valores_posibles) for _ in range(self.longitud_cromosoma)]
            poblacion.append(cromosoma)
        return poblacion
    
    def evaluar_poblacion(self):
        fitnesses = []
        for cromosoma in self.poblacion:
            fitness = self.funcion_fitness(cromosoma)
            fitnesses.append(fitness)
        return fitnesses
    
    def seleccionar(self, fitnesses):
        poblacion_ordenada = sorted(zip(self.poblacion, fitnesses), key=lambda x: x[1], reverse=True)
        mejores_parejas = [cromosoma for cromosoma, _ in poblacion_ordenada[:len(self.poblacion)//2]]
        
        if len(mejores_parejas) % 2 != 0:
            mejores_parejas.append(mejores_parejas[-1])
        
        return mejores_parejas
    
    #Implementación de la función de cruce no destructiva
    def cruzar(self, mejores_parejas):
        nueva_poblacion = []
        for i in range(0, len(mejores_parejas), 2):
            cromosoma1 = mejores_parejas[i]
            cromosoma2 = mejores_parejas[i + 1]
            hijos = self.funcion_cruzar(cromosoma1, cromosoma2)
            
            # Calcula el fitness de los hijos y de los padres
            fitness_padre1 = self.funcion_fitness(cromosoma1)
            fitness_padre2 = self.funcion_fitness(cromosoma2)
            fitness_hijo1 = self.funcion_fitness(hijos[0])
            fitness_hijo2 = self.funcion_fitness(hijos[1])
            
            # Solo se conserva el hijo si tiene mejor fitness que ambos padres
            if fitness_hijo1 > fitness_padre1 and fitness_hijo1 > fitness_padre2:
                nueva_poblacion.append(hijos[0])
            else:
                nueva_poblacion.append(cromosoma1) #si el hijo1 no supera el fitness de ambos padres, se conserva el padre1
            
            if fitness_hijo2 > fitness_padre1 and fitness_hijo2 > fitness_padre2:
                nueva_poblacion.append(hijos[1])
            else:
                nueva_poblacion.append(cromosoma2) #si el hijo2 no supera el fitness de ambos padres, se conserva el padre2
        
        return nueva_poblacion
    
    def mutar(self, nueva_poblacion, prob_mutacion):
        return [self.funcion_mutar(cromosoma, prob_mutacion) for cromosoma in nueva_poblacion]
    
    def resolver(self, generaciones, prob_mutacion):
        for generacion in range(generaciones):
            print(f"\n--- Generación {generacion + 1} ---")
            fitnesses = self.evaluar_poblacion()
            print(f"Fitnesses de la población: {fitnesses}")
            print(f"Mejor fitness en esta generación: {max(fitnesses)}")
            
            mejores_parejas = self.seleccionar(fitnesses)
            nueva_poblacion = self.cruzar(mejores_parejas)
            nueva_poblacion = self.mutar(nueva_poblacion, prob_mutacion)
            
            self.poblacion = nueva_poblacion[:self.tamano_poblacion]

        # Evaluamos la población final
        fitnesses = self.evaluar_poblacion()
        mejor_fitness = max(fitnesses)
        indice_mejor = fitnesses.index(mejor_fitness)
        mejor_cromosoma = self.poblacion[indice_mejor]
        print("\nMejor cromosoma final:")
        print(f"Cromosoma: {mejor_cromosoma} - Fitness: {mejor_fitness}")


longitud_cromosoma = 43
valores_posibles = [0, 1]  # Suposición de valores posibles para los cromosomas (0 o 1)
problema = ProblemaGenetico(valores_posibles, binario_a_decimal_rango, fun_mutar, fun_cruzar, fun_fitness, 10, longitud_cromosoma)

# Resolver el problema durante 100 generaciones con probabilidad de mutación de 0.1
problema.resolver(100, 0.1)


En esta nueva implementación de la función de cruce, se exige que el fitnes de los hijos sea estrictamente mayor al de los padres. Sin embargo, siguen sin pasar los dos mejores cromosomas: por la construcción del código es perfectamente posible que pase el hijo1 al ser mayor que sus dos padres pero, si el hijo2 fuese más pequeño que sus antecesores, pasará el padre2 aunque este tenga un fitness menor que el padre1.

Se mantiene esa posibilidad para evitar que sea demasiado elitista (ya que la función de selección ya lo es) en la construcción de población y sea probable rescatar partes buenas de cromosomas 'malos', teniendo en cuenta que la función de selección ya va a darnos las mejores parejas.

Se incluye, sin embargo, una probabilidad de cruce, lo que hará que sea aún más complicado el paso de cromosomas realmente malos, aunque esta probabilidad de cruce será alta.

In [ ]:
import random

# Función para convertir un cromosoma binario a un número decimal con un rango específico
def binario_a_decimal_rango(cromosoma, min_valor, max_valor):
    decimal = int("".join(str(x) for x in cromosoma), 2)
    rango = max_valor - min_valor
    valor = min_valor + (decimal % (rango + 1))
    return valor

# Función de cruce
def fun_cruzar(cromosoma1, cromosoma2):
    l1 = len(cromosoma1)
    l2 = len(cromosoma2)
    cruce1 = cromosoma1[0:l1//2] + cromosoma2[l1//2:l2]
    cruce2 = cromosoma2[0:l2//2] + cromosoma1[l2//2:l1]
    return [cruce1, cruce2]

# Función de mutación
def fun_mutar(cromosoma, prob):
    for i in range(len(cromosoma)):
        if random.uniform(0, 1) < prob:
            cromosoma[i] = 1 - cromosoma[i]  # Cambia el bit de 0 a 1 o de 1 a 0
    return cromosoma

# Función de fitness
def fun_fitness(cromosoma):
    # Decodificación de los parámetros
    omega = binario_a_decimal_rango(cromosoma[0:10], 0, 200)  # Omega (rad/s)
    R = binario_a_decimal_rango(cromosoma[10:20], 0.1, 2)     # Radio (m)
    b = binario_a_decimal_rango(cromosoma[20:22], 2, 5)       # Número de palas
    theta0 = binario_a_decimal_rango(cromosoma[22:27], -0.26, 0.26)  # Ángulo de paso colectivo (radianes)
    p = binario_a_decimal_rango(cromosoma[27:35], 0, 25)      # Torsión (grados)
    cuerda = binario_a_decimal_rango(cromosoma[35:43], 0.01, 0.2)  # Anchura de la pala (m)
    
    # Valores fijos
    vz = 70  # Velocidad de vuelo fija (m/s)
    h = 1000  # Altura fija (m)

    print(f"\nEvaluando cromosoma: {cromosoma}")
    print(f"Parámetros decodificados: omega={omega}, R={R}, b={b}, theta0={theta0}, p={p}, cuerda={cuerda}")

    try:
        # Calcular hélice con los parámetros decodificados
        T, P, efic, mach_punta = calcular_helice(omega, vz, R, b, h, theta0=theta0, tors_param=['h',p], chord_params=cuerda)
        print(f"Resultados de calcular_helice: T={T}, P={P}, efic={efic}, mach_punta={mach_punta}")
       
        # Fitness basado en maximizar la eficiencia
        fitness = efic*10 # Calculamos el fitness como la propia eficiencia, se ha multiplicado para que no salgan valores muy pequeños (por comodidad)
        print(f"Fitness calculado: {fitness}")
    except Exception as e:
        print(f"Error en calcular_helice: {e}")
        fitness = 0
    return fitness


class ProblemaGenetico:
    def __init__(self, valores_posibles, funcion_decodificar, funcion_mutar, funcion_cruzar, funcion_fitness, tamano_poblacion, longitud_cromosoma):
        self.valores_posibles = valores_posibles
        self.funcion_decodificar = funcion_decodificar
        self.funcion_mutar = funcion_mutar
        self.funcion_cruzar = funcion_cruzar
        self.funcion_fitness = funcion_fitness
        self.tamano_poblacion = tamano_poblacion
        self.longitud_cromosoma = longitud_cromosoma  
        self.poblacion = self.crear_poblacion()
    
    def crear_poblacion(self):
        poblacion = []
        for _ in range(self.tamano_poblacion):
            cromosoma = [random.choice(self.valores_posibles) for _ in range(self.longitud_cromosoma)]
            poblacion.append(cromosoma)
        return poblacion
    
    def evaluar_poblacion(self):
        fitnesses = []
        for cromosoma in self.poblacion:
            fitness = self.funcion_fitness(cromosoma)
            fitnesses.append(fitness)
        return fitnesses
    
    def seleccionar(self, fitnesses):
        poblacion_ordenada = sorted(zip(self.poblacion, fitnesses), key=lambda x: x[1], reverse=True)
        mejores_parejas = [cromosoma for cromosoma, _ in poblacion_ordenada[:len(self.poblacion)//2]]
        
        if len(mejores_parejas) % 2 != 0:
            mejores_parejas.append(mejores_parejas[-1])
        
        return mejores_parejas
    
    #Implementación de la función de cruce no destructiva y con probabilidad de cruce
    def cruzar(self, mejores_parejas, prob_cruce):
        nueva_poblacion = []
        for i in range(0, len(mejores_parejas), 2):
            cromosoma1 = mejores_parejas[i]
            cromosoma2 = mejores_parejas[i + 1]
            # Determina si ocurre el cruce
            if random.uniform(0, 1) < prob_cruce:
                hijos = self.funcion_cruzar(cromosoma1, cromosoma2)
                
                # Calcula el fitness de los hijos y de los padres
                fitness_padre1 = self.funcion_fitness(cromosoma1)
                fitness_padre2 = self.funcion_fitness(cromosoma2)
                fitness_hijo1 = self.funcion_fitness(hijos[0])
                fitness_hijo2 = self.funcion_fitness(hijos[1])
                
                # Solo se conserva el hijo si tiene mejor fitness que ambos padres
                if fitness_hijo1 > fitness_padre1 and fitness_hijo1 > fitness_padre2:
                    nueva_poblacion.append(hijos[0])
                else:
                    nueva_poblacion.append(cromosoma1) #si el hijo1 no supera el fitness de ambos padres, se conserva el padre1
                
                if fitness_hijo2 > fitness_padre1 and fitness_hijo2 > fitness_padre2:
                    nueva_poblacion.append(hijos[1])
                else:
                    nueva_poblacion.append(cromosoma2) #si el hijo2 no supera el fitness de ambos padres, se conserva el padre2
            
            else:
                # Si no ocurre cruce, se copian los padres directamente
                nueva_poblacion.append(cromosoma1)
                nueva_poblacion.append(cromosoma2)
        
        return nueva_poblacion
    
    def mutar(self, nueva_poblacion, prob_mutacion):
        return [self.funcion_mutar(cromosoma, prob_mutacion) for cromosoma in nueva_poblacion]
    
    def resolver(self, generaciones, prob_mutacion, prob_cruce):
        for generacion in range(generaciones):
            print(f"\n--- Generación {generacion + 1} ---")
            fitnesses = self.evaluar_poblacion()
            print(f"Fitnesses de la población: {fitnesses}")
            print(f"Mejor fitness en esta generación: {max(fitnesses)}")
            
            mejores_parejas = self.seleccionar(fitnesses)
            nueva_poblacion = self.cruzar(mejores_parejas, prob_cruce)
            nueva_poblacion = self.mutar(nueva_poblacion, prob_mutacion)
            
            self.poblacion = nueva_poblacion[:self.tamano_poblacion]

        # Evaluamos la población final
        fitnesses = self.evaluar_poblacion()
        mejor_fitness = max(fitnesses)
        indice_mejor = fitnesses.index(mejor_fitness)
        mejor_cromosoma = self.poblacion[indice_mejor]
        print("\nMejor cromosoma final:")
        print(f"Cromosoma: {mejor_cromosoma} - Fitness: {mejor_fitness}")


longitud_cromosoma = 43
valores_posibles = [0, 1]  # Suposición de valores posibles para los cromosomas (0 o 1)
problema = ProblemaGenetico(valores_posibles, binario_a_decimal_rango, fun_mutar, fun_cruzar, fun_fitness, 10, longitud_cromosoma)

# Resolver el problema durante 100 generaciones con probabilidad de mutación de 0.1 y probabilidad de cruce de 0.8
problema.resolver(100, prob_mutacion=0.1, prob_cruce=0.8)


La nueva implementación de la función de cruce con la probabilidad hace que los padres tengan aún más posibilidades de sobrevivir en cada paso de generación. Esto puede parecer útil si se encuentran padres con fitness alto al principio del análisis y nuestro objetivo principal es mantenerlo pero plantea otro problema: falta de diversidad genética.

La selección elitista, la función de cruce no destructiva y, además, la probabilidad de cruce hace que se generen mesetas en cuanto a la obtención de poblaciones nuevas se refiere.

Una primera solución sería volver a modificar la función de cruce haciendo que los cromosomas se recombinen por dos puntos en vez de por uno solo que además siempre era por el medio y no lo elegía aleatoriamente.

In [ ]:
import random

# Función para convertir un cromosoma binario a un número decimal con un rango específico
def binario_a_decimal_rango(cromosoma, min_valor, max_valor):
    decimal = int("".join(str(x) for x in cromosoma), 2)
    rango = max_valor - min_valor
    valor = min_valor + (decimal % (rango + 1))
    return valor

# Función de cruce por dos puntos elegidos aleatoriamente
def fun_cruzar(cromosoma1, cromosoma2):
    punto1 = random.randint(1, len(cromosoma1) - 2)
    punto2 = random.randint(punto1, len(cromosoma1) - 1)
    cruce1 = cromosoma1[:punto1] + cromosoma2[punto1:punto2] + cromosoma1[punto2:]
    cruce2 = cromosoma2[:punto1] + cromosoma1[punto1:punto2] + cromosoma2[punto2:]
    return [cruce1, cruce2]

# Función de mutación
def fun_mutar(cromosoma, prob):
    for i in range(len(cromosoma)):
        if random.uniform(0, 1) < prob:
            cromosoma[i] = 1 - cromosoma[i]  # Cambia el bit de 0 a 1 o de 1 a 0
    return cromosoma

# Función de fitness
def fun_fitness(cromosoma):
    # Decodificación de los parámetros
    omega = binario_a_decimal_rango(cromosoma[0:10], 0, 200)  # Omega (rad/s)
    R = binario_a_decimal_rango(cromosoma[10:20], 0.1, 2)     # Radio (m)
    b = binario_a_decimal_rango(cromosoma[20:22], 2, 5)       # Número de palas
    theta0 = binario_a_decimal_rango(cromosoma[22:27], -0.26, 0.26)  # Ángulo de paso colectivo (radianes)
    p = binario_a_decimal_rango(cromosoma[27:35], 0, 25)      # Torsión (grados)
    cuerda = binario_a_decimal_rango(cromosoma[35:43], 0.01, 0.2)  # Anchura de la pala (m)
    
    # Valores fijos
    vz = 70  # Velocidad de vuelo fija (m/s)
    h = 1000  # Altura fija (m)

    print(f"\nEvaluando cromosoma: {cromosoma}")
    print(f"Parámetros decodificados: omega={omega}, R={R}, b={b}, theta0={theta0}, p={p}, cuerda={cuerda}")

    try:
        # Calcular hélice con los parámetros decodificados
        T, P, efic, mach_punta = calcular_helice(omega, vz, R, b, h, theta0=theta0, tors_param=['h',p], chord_params=cuerda)
        print(f"Resultados de calcular_helice: T={T}, P={P}, efic={efic}, mach_punta={mach_punta}")
       
        # Fitness basado en maximizar la eficiencia
        fitness = efic*10 # Calculamos el fitness como la propia eficiencia, se ha multiplicado para que no salgan valores muy pequeños (por comodidad)
        print(f"Fitness calculado: {fitness}")
    except Exception as e:
        print(f"Error en calcular_helice: {e}")
        fitness = 0
    return fitness


class ProblemaGenetico:
    def __init__(self, valores_posibles, funcion_decodificar, funcion_mutar, funcion_cruzar, funcion_fitness, tamano_poblacion, longitud_cromosoma):
        self.valores_posibles = valores_posibles
        self.funcion_decodificar = funcion_decodificar
        self.funcion_mutar = funcion_mutar
        self.funcion_cruzar = funcion_cruzar
        self.funcion_fitness = funcion_fitness
        self.tamano_poblacion = tamano_poblacion
        self.longitud_cromosoma = longitud_cromosoma  
        self.poblacion = self.crear_poblacion()
    
    def crear_poblacion(self):
        poblacion = []
        for _ in range(self.tamano_poblacion):
            cromosoma = [random.choice(self.valores_posibles) for _ in range(self.longitud_cromosoma)]
            poblacion.append(cromosoma)
        return poblacion
    
    def evaluar_poblacion(self):
        fitnesses = []
        for cromosoma in self.poblacion:
            fitness = self.funcion_fitness(cromosoma)
            fitnesses.append(fitness)
        return fitnesses
    
    def seleccionar(self, fitnesses):
        poblacion_ordenada = sorted(zip(self.poblacion, fitnesses), key=lambda x: x[1], reverse=True)
        mejores_parejas = [cromosoma for cromosoma, _ in poblacion_ordenada[:len(self.poblacion)//2]]
        
        if len(mejores_parejas) % 2 != 0:
            mejores_parejas.append(mejores_parejas[-1])
        
        return mejores_parejas
    
    #Implementación de la función de cruce no destructiva y con probabilidad de cruce
    def cruzar(self, mejores_parejas, prob_cruce):
        nueva_poblacion = []
        for i in range(0, len(mejores_parejas), 2):
            cromosoma1 = mejores_parejas[i]
            cromosoma2 = mejores_parejas[i + 1]
            # Determina si ocurre el cruce
            if random.uniform(0, 1) < prob_cruce:
                hijos = self.funcion_cruzar(cromosoma1, cromosoma2)
                
                # Calcula el fitness de los hijos y de los padres
                fitness_padre1 = self.funcion_fitness(cromosoma1)
                fitness_padre2 = self.funcion_fitness(cromosoma2)
                fitness_hijo1 = self.funcion_fitness(hijos[0])
                fitness_hijo2 = self.funcion_fitness(hijos[1])
                
                # Solo se conserva el hijo si tiene mejor fitness que ambos padres
                if fitness_hijo1 > fitness_padre1 and fitness_hijo1 > fitness_padre2:
                    nueva_poblacion.append(hijos[0])
                else:
                    nueva_poblacion.append(cromosoma1) #si el hijo1 no supera el fitness de ambos padres, se conserva el padre1
                
                if fitness_hijo2 > fitness_padre1 and fitness_hijo2 > fitness_padre2:
                    nueva_poblacion.append(hijos[1])
                else:
                    nueva_poblacion.append(cromosoma2) #si el hijo2 no supera el fitness de ambos padres, se conserva el padre2
            
            else:
                # Si no ocurre cruce, se copian los padres directamente
                nueva_poblacion.append(cromosoma1)
                nueva_poblacion.append(cromosoma2)
        
        return nueva_poblacion
    
    def mutar(self, nueva_poblacion, prob_mutacion):
        return [self.funcion_mutar(cromosoma, prob_mutacion) for cromosoma in nueva_poblacion]
    
    def resolver(self, generaciones, prob_mutacion, prob_cruce):
        for generacion in range(generaciones):
            print(f"\n--- Generación {generacion + 1} ---")
            fitnesses = self.evaluar_poblacion()
            print(f"Fitnesses de la población: {fitnesses}")
            print(f"Mejor fitness en esta generación: {max(fitnesses)}")
            
            mejores_parejas = self.seleccionar(fitnesses)
            nueva_poblacion = self.cruzar(mejores_parejas, prob_cruce)
            nueva_poblacion = self.mutar(nueva_poblacion, prob_mutacion)
            
            self.poblacion = nueva_poblacion[:self.tamano_poblacion]

        # Evaluamos la población final
        fitnesses = self.evaluar_poblacion()
        mejor_fitness = max(fitnesses)
        indice_mejor = fitnesses.index(mejor_fitness)
        mejor_cromosoma = self.poblacion[indice_mejor]
        print("\nMejor cromosoma final:")
        print(f"Cromosoma: {mejor_cromosoma} - Fitness: {mejor_fitness}")


longitud_cromosoma = 43
valores_posibles = [0, 1]  # Suposición de valores posibles para los cromosomas (0 o 1)
problema = ProblemaGenetico(valores_posibles, binario_a_decimal_rango, fun_mutar, fun_cruzar, fun_fitness, 10, longitud_cromosoma)

# Resolver el problema durante 100 generaciones con probabilidad de mutación de 0.1 y probabilidad de cruce de 0.8
problema.resolver(100, prob_mutacion=0.1, prob_cruce=0.8)


Aún así, para introducir una mayor variedad genética la solución pasa por modificar la función de selección ya que resulta demasiado elitista y no da pie a que cromsomas con poco fitness puedan sobrevivir y aportar diferentes genes en las posteriores generaciones.

Se implemementa una función de selección por torneo.

In [ ]:
import random

# Función para convertir un cromosoma binario a un número decimal con un rango específico
def binario_a_decimal_rango(cromosoma, min_valor, max_valor):
    decimal = int("".join(str(x) for x in cromosoma), 2)
    rango = max_valor - min_valor
    valor = min_valor + (decimal % (rango + 1))
    return valor

# Función de cruce por dos puntos elegidos aleatoriamente
def fun_cruzar(cromosoma1, cromosoma2):
    punto1 = random.randint(1, len(cromosoma1) - 2)
    punto2 = random.randint(punto1, len(cromosoma1) - 1)
    cruce1 = cromosoma1[:punto1] + cromosoma2[punto1:punto2] + cromosoma1[punto2:]
    cruce2 = cromosoma2[:punto1] + cromosoma1[punto1:punto2] + cromosoma2[punto2:]
    return [cruce1, cruce2]

# Función de mutación
def fun_mutar(cromosoma, prob):
    for i in range(len(cromosoma)):
        if random.uniform(0, 1) < prob:
            cromosoma[i] = 1 - cromosoma[i]  # Cambia el bit de 0 a 1 o de 1 a 0
    return cromosoma

# Función de fitness
def fun_fitness(cromosoma):
    # Decodificación de los parámetros
    omega = binario_a_decimal_rango(cromosoma[0:10], 0, 200)  # Omega (rad/s)
    R = binario_a_decimal_rango(cromosoma[10:20], 0.1, 2)     # Radio (m)
    b = binario_a_decimal_rango(cromosoma[20:22], 2, 5)       # Número de palas
    theta0 = binario_a_decimal_rango(cromosoma[22:27], -0.26, 0.26)  # Ángulo de paso colectivo (radianes)
    p = binario_a_decimal_rango(cromosoma[27:35], 0, 25)      # Torsión (grados)
    cuerda = binario_a_decimal_rango(cromosoma[35:43], 0.01, 0.2)  # Anchura de la pala (m)
    
    # Valores fijos
    vz = 70  # Velocidad de vuelo fija (m/s)
    h = 1000  # Altura fija (m)

    print(f"\nEvaluando cromosoma: {cromosoma}")
    print(f"Parámetros decodificados: omega={omega}, R={R}, b={b}, theta0={theta0}, p={p}, cuerda={cuerda}")

    try:
        # Calcular hélice con los parámetros decodificados
        T, P, efic, mach_punta = calcular_helice(omega, vz, R, b, h, theta0=theta0, tors_param=['h',p], chord_params=cuerda)
        print(f"Resultados de calcular_helice: T={T}, P={P}, efic={efic}, mach_punta={mach_punta}")
       
        # Fitness basado en maximizar la eficiencia
        fitness = efic*10 # Calculamos el fitness como la propia eficiencia, se ha multiplicado para que no salgan valores muy pequeños (por comodidad)
        print(f"Fitness calculado: {fitness}")
    except Exception as e:
        print(f"Error en calcular_helice: {e}")
        fitness = 0
    return fitness


class ProblemaGenetico:
    def __init__(self, valores_posibles, funcion_decodificar, funcion_mutar, funcion_cruzar, funcion_fitness, tamano_poblacion, longitud_cromosoma):
        self.valores_posibles = valores_posibles
        self.funcion_decodificar = funcion_decodificar
        self.funcion_mutar = funcion_mutar
        self.funcion_cruzar = funcion_cruzar
        self.funcion_fitness = funcion_fitness
        self.tamano_poblacion = tamano_poblacion
        self.longitud_cromosoma = longitud_cromosoma  
        self.poblacion = self.crear_poblacion()
    
    def crear_poblacion(self):
        poblacion = []
        for _ in range(self.tamano_poblacion):
            cromosoma = [random.choice(self.valores_posibles) for _ in range(self.longitud_cromosoma)]
            poblacion.append(cromosoma)
        return poblacion
    
    def evaluar_poblacion(self):
        fitnesses = []
        for cromosoma in self.poblacion:
            fitness = self.funcion_fitness(cromosoma)
            fitnesses.append(fitness)
        return fitnesses
    
    #Implementación de la función de selección por torneo
    def seleccionar_por_torneo(self, fitnesses, tamano_torneo):
        seleccionados = []
        for _ in range(self.tamano_poblacion):
            indices_torneo = random.sample(range(len(self.poblacion)), tamano_torneo)
            participantes = [(self.poblacion[i], fitnesses[i]) for i in indices_torneo]
            ganador = max(participantes, key=lambda x: x[1])[0]
            seleccionados.append(ganador)
        return seleccionados
    
    def seleccionar(self, fitnesses, tamano_torneo=3):
        return self.seleccionar_por_torneo(fitnesses, 3)
    
    #Implementación de la función de cruce no destructiva y con probabilidad de cruce
    def cruzar(self, mejores_parejas, prob_cruce):
        nueva_poblacion = []
        for i in range(0, len(mejores_parejas), 2):
            cromosoma1 = mejores_parejas[i]
            cromosoma2 = mejores_parejas[i + 1]
            # Determina si ocurre el cruce
            if random.uniform(0, 1) < prob_cruce:
                hijos = self.funcion_cruzar(cromosoma1, cromosoma2)
                
                # Calcula el fitness de los hijos y de los padres
                fitness_padre1 = self.funcion_fitness(cromosoma1)
                fitness_padre2 = self.funcion_fitness(cromosoma2)
                fitness_hijo1 = self.funcion_fitness(hijos[0])
                fitness_hijo2 = self.funcion_fitness(hijos[1])
                
                # Solo se conserva el hijo si tiene mejor fitness que ambos padres
                if fitness_hijo1 > fitness_padre1 and fitness_hijo1 > fitness_padre2:
                    nueva_poblacion.append(hijos[0])
                else:
                    nueva_poblacion.append(cromosoma1) #si el hijo1 no supera el fitness de ambos padres, se conserva el padre1
                
                if fitness_hijo2 > fitness_padre1 and fitness_hijo2 > fitness_padre2:
                    nueva_poblacion.append(hijos[1])
                else:
                    nueva_poblacion.append(cromosoma2) #si el hijo2 no supera el fitness de ambos padres, se conserva el padre2
            
            else:
                # Si no ocurre cruce, se copian los padres directamente
                nueva_poblacion.append(cromosoma1)
                nueva_poblacion.append(cromosoma2)
        
        return nueva_poblacion
    
    def mutar(self, nueva_poblacion, prob_mutacion):
        return [self.funcion_mutar(cromosoma, prob_mutacion) for cromosoma in nueva_poblacion]
    
    def resolver(self, generaciones, prob_mutacion, prob_cruce):
        for generacion in range(generaciones):
            print(f"\n--- Generación {generacion + 1} ---")
            fitnesses = self.evaluar_poblacion()
            print(f"Fitnesses de la población: {fitnesses}")
            print(f"Mejor fitness en esta generación: {max(fitnesses)}")
            
            mejores_parejas = self.seleccionar(fitnesses)
            nueva_poblacion = self.cruzar(mejores_parejas, prob_cruce)
            nueva_poblacion = self.mutar(nueva_poblacion, prob_mutacion)
            
            self.poblacion = nueva_poblacion[:self.tamano_poblacion]

        # Evaluamos la población final
        fitnesses = self.evaluar_poblacion()
        mejor_fitness = max(fitnesses)
        indice_mejor = fitnesses.index(mejor_fitness)
        mejor_cromosoma = self.poblacion[indice_mejor]
        print("\nMejor cromosoma final:")
        print(f"Cromosoma: {mejor_cromosoma} - Fitness: {mejor_fitness}")


longitud_cromosoma = 43
valores_posibles = [0, 1]  # Suposición de valores posibles para los cromosomas (0 o 1)
problema = ProblemaGenetico(valores_posibles, binario_a_decimal_rango, fun_mutar, fun_cruzar, fun_fitness, 10, longitud_cromosoma)

# Resolver el problema durante 100 generaciones con probabilidad de mutación de 0.1 y probabilidad de cruce de 0.8
problema.resolver(100, prob_mutacion=0.1, prob_cruce=0.8)


En la nueva selección por torneo se escogen conjuntos aleatorios de 3 participantes y se elige el ganador (aquel con mayor fitness). Esta supone una myor diversidad genética ya que no se va a seleccionar a los cromosomas con mayor fitness de toda la población, sino a aquel que tenga mayor fitness que unos pocos.

Se consigue de esta manera que haya posibilidad de que cromosomas no tan 'buenos' puedan aportar en el paso de las generaciones pero prevalezcan aquellos con un fitness aceptable y no se renuncie a la parte de optimización que es de lo que se trata. 

Por otro lado, una mayor probabilidad de mutación también aumentará la diversidad aunque de forma menos controlada.

#### CONCLUSIONES:

- Sin una representación adecuada del cromosoma no sería posible una implementación buena. Es crucial dar a cada variable un número de bits adecuado y más en este caso al tratarse de números decimales que pueden tomar un infinito rango de valores.

- La función de fitness es quizás la que más impacto tenga en el desarrollo de la implementación del problema. Nos sirve de guía para el resto de funciones a implementar y, al final, todo viene dado por el fitness. De haberlas, es muy importante tener claras las penalizaciones para implementarlas de la mejor manera: viendo que aspectos debemos penalizar más o menos.

- La función de cruce resulta clave si queremos avanzar en cada generación y recombinar los cromosomas de manera que no se estanque demasiado en la obtención de hijos y terminen siendo iguales que los padres al tiempo que damos importancia a la optimización haciendo que los cromosomas con mejor fitness prevalezcan.

- La función de selección también juega su papel en el hecho de no renunciar a crear nuevas generaciones diversas y obtener combinaciones que nos lleven a un 'buen' cromosoma.

- En cuanto a la función de mutación, nos decantamos por mantener una probabilidad baja y hacer que la mutación no tenga un gran impacto en la generación de hijos. Debe mantenerse como un fenómeno raro. Sin embargo, sí creemos que puede resultar útil en algunas ocasiones y aporta ese punto de azar.

Sin duda lo más importante es mantener un equilibrio entre avanzar hacia obtener un fitness finalmente muy alto sin renunciar a la variedad de cromosomas y genes en el proceso, ya que esta nos puede llevar a mejores resultados que simplemente quedarnos con lo mejor en cada paso.